<a href="https://colab.research.google.com/github/MarathaAshhhh/Events_Analysis/blob/main/Final_crypto_attended_events.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install tensorflow_text==2.8.*

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 14.6 MB/s 
     |████████████████████████████████| 497.9 MB 32 kB/s 
     |████████████████████████████████| 462 kB 65.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.10.0
    Uninstalling tensorflow-2.10.0:
      Successfully uninstalled tensorflow-2.10.0
  Attempting uninstall: tensorflow-text
    Found existing installation: tensorflow-text 2.10.0
    Uninstalling tensorflow-text-2.10.0:
      Successfully uninstalled tensorflow-text-2.10.0


In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [2]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/final_filtered_events.csv")
df.head(5)

,Unnamed: 0,Tweet,Attending_index
0,0,"Well, I look forward to attending those. Just ...",0
1,1,"Unfortunately, due to work commitments Im unab...",0
2,2,PrepHoopsTN Combine Campers: Our event is movi...,0
3,3,Community (in one form or another) and sharing...,0
4,4,Im fortunate that I wont be in trouble financi...,0


In [5]:
df.groupby('Attending_index').describe()

Unnamed: 0                                                 \
                     count         mean          std  min     25%     50%   
Attending_index                                                             
0                   3567.0  3649.677320  2077.366104  0.0  1843.5  3629.0   
1                   3671.0  3588.205938  2101.209754  5.0  1780.0  3602.0   

                                 
                    75%     max  
Attending_index                  
0                5465.0  7235.0  
1                5384.0  7237.0

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Tweet'],df['Attending_index'], stratify=df['Attending_index'])

In [4]:
X_train.head(4)

2563    Announcement: Sadly, due to feeling very under...
6809    Never said it was Flash's money at all. I'm sa...
6443    Seems like everyone I talk to lately in the me...
243     Many huskies love the water. I attend meetups ...
Name: Tweet, dtype: object

In [5]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [6]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)


<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351724, -0.51327276, -0.88845724, ..., -0.74748844,
        -0.7531474 ,  0.91964495],
       [-0.87208337, -0.5054394 , -0.94446665, ..., -0.85847497,
        -0.71745324,  0.8808295 ]], dtype=float32)>

In [7]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9911088]], dtype=float32)

In [9]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)


model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [10]:
len(X_train)

5428

In [12]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [13]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
170/170 [==============================] - 67s 342ms/step - loss: 0.6705 - accuracy: 0.5847 - precision: 0.5903 - recall: 0.5924
Epoch 2/10
170/170 [==============================] - 56s 332ms/step - loss: 0.6247 - accuracy: 0.6606 - precision: 0.6622 - recall: 0.6756
Epoch 3/10
170/170 [==============================] - 57s 334ms/step - loss: 0.5886 - accuracy: 0.6975 - precision: 0.7021 - recall: 0.7011
Epoch 4/10
170/170 [==============================] - 57s 337ms/step - loss: 0.5703 - accuracy: 0.7163 - precision: 0.7142 - recall: 0.7345
Epoch 5/10
170/170 [==============================] - 56s 331ms/step - loss: 0.5470 - accuracy: 0.7395 - precision: 0.7434 - recall: 0.7428
Epoch 6/10
170/170 [==============================] - 57s 333ms/step - loss: 0.5337 - accuracy: 0.7533 - precision: 0.7518 - recall: 0.7668
Epoch 7/10
170/170 [==============================] - 57s 334ms/step - loss: 0.5186 - accuracy: 0.7522 - precision: 0.7491 - recall: 0.7690
Epoch 8/10
170/170 [

In [14]:
model.evaluate(X_test, y_test)

57/57 [==============================] - 20s 325ms/step - loss: 0.4601 - accuracy: 0.8066 - precision: 0.8287 - recall: 0.7800


[0.46010953187942505,
 0.8066298365592957,
 0.8287037014961243,
 0.7799564003944397]

In [33]:
!pip install h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
model.save_weights("model_new.h5")